In [2]:
import os
from typing import Any
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time

# 准备数据
def read_file(path, label=True):
    """读取图片文件"""
    image_dir = sorted(os.listdir(path))
    # 图片数据形式，3为rgb3通道
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    # 遍历列表
    for i, file in enumerate(image_dir):
        # 原图片是512*512，重新调整大小为128*128
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img, (128, 128))
        # 若包含分类信息，则设置分类信息
        if label:
            y[i] = int(file.split("_")[0])
    if label:
        return x, y
    else:
        return x


train_x, train_y = read_file("./resource/training")
val_x, val_y = read_file("./resource/validation")
test_x = read_file("./resource/testing", False)


In [3]:
# 预处理图片数据
train_transform=transforms.Compose([
    # PIL图像 python image library，python的知名第三方图形处理库
    transforms.ToPILImage(),
    # 随机水平翻转图片
    transforms.RandomHorizontalFlip(),
    # 随机旋转图片
    transforms.RandomRotation(15),
    # 将图片转为pytorch的tensor，并且自动把数值normalize到[0,1]
    transforms.ToTensor(),
])

test_transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

# 继承于Dataset
class ImageDataset(Dataset):
    # 构造函数，python没有提供默认的this指针，需要自动手动传一个self
    def __init__(self,x,y=None,transform=None):
        self.x=x
        self.y=y
        if y is not None:
            # 将numpy array格式的y转为pytorch的tensor
            self.y=torch.LongTensor(y)
        self.transform=transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        _x=self.x[index]
        if self.transform is not None:
            _x=self.transform(_x)
        if self.y is not None:
            _y=self.y[index]
            return _x,_y
        else:
            return _x

In [4]:
batch_size=128
train_set=ImageDataset(train_x,train_y,train_transform)
val_set=ImageDataset(val_x,val_y,test_transform)
# dataloader：数据迭代器，实现批量(batch)读取，打乱数据(shuffle)，并提供并行加速等功能
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_set,batch_size=batch_size,shuffle=False)

In [ ]:
# model
# 分类器，继承于nn.Module
# 5层卷积层，3层全连接层，这里应该是AlexNet模型
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        self.cnn=nn.Sequential(
            # 卷积层，所以此处用的是cnn，作用可能是匹配出某一模式(比如检测是否是连续的，方向是向上还是向右的)，相对传统全连接神经网络，好处是参数少，计算量小，不容易过拟合
            # 最开始输入维度是3*128*128，这里输出维度为64，应该是会自动算出所要求的卷积核的规模（最终目的是计算出这些卷积核或过滤器的数值）
            nn.Conv2d(3,64,3,1,1),
            # 归一化处理，这使得数据在进行Relu之前不会因为数据过大而导致网络性能的不稳定
            nn.BatchNorm2d(64),
            # 激活函数用ReLU，一般比sigmoid好
            nn.ReLU(),
            # 进行MaxPool进行池化，又称下采样，让神经网络拥有抽象能力？
            # 池化层在CNN中可用来减小尺寸，提高运算速度及减小噪声影响，让各特征更具有健壮性。池化层比卷积层更简单，它没有卷积运算，只是在滤波器算子滑动区域内取最大值或平均值。而池化的作用则体现在降采样：保留显著特征、降低特征维度，增大感受野。深度网络越往后面越能捕捉到物体的语义信息，这种语义信息是建立在较大的感受野基础上
            nn.MaxPool2d(2,2,0),

            nn.Conv2d(64,128,3,1,1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0),

            nn.Conv2d(128,256,3,1,1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0),

            nn.Conv2d(256,512,3,1,1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0),

            nn.Conv2d(512,512,3,1,1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0), #最后得到的结果是512*4*4
        )

        # flatten，拉直
        self.fc=nn.Sequential(
            # 全连接层
            # 输入是512*4*4，输出1024
            nn.Linear(512*4*4,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            # 最终拉成11分类的结果
            nn.Linear(512,11),
        )

    def forward(self, x):
        out=self.cnn(x)
        out=out.view(out.size()[0],-1)
        return self.fc(out)

# 训练(使用gpu)
model=Classifier().cuda()
# 因为是分类问题，所以loss函数使用CrossEntropy
loss=nn.CrossEntropyLoss()
# 优化使用adam
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
num_epoch=30

for epoch in range(num_epoch):
    epoch_start_time=time.time()
    train_acc=0.0
    train_loss=0.0
    val_acc=0.0
    val_loss=0.0
    # 相关资料可参考：https://www.cnblogs.com/pinard/p/6418668.html
    # 层、模型、损失函数和优化器等都定义或创建好，接下来就是训练模型。训练模型时需要注意使模型处于训练模式，即调用model.train()。调用 model.train()会把所有的module设置为训练模式。如果是测试或验证阶段， 需要使模型处于验证阶段，即调用model.eval()，调用model.eval()会把所有的training属性设置为False。缺省情况下梯度是累加的，需要手工把梯度初始化或清零，调用optimizer.zero_grad()即可。训练过程中，正向传播生成网络的输出，计算输 出和实际值之间的损失值。调用loss.backward()自动生成梯度，然后使用 optimizer.step() 执行优化器，把梯度传播回每个网络
    model.train()
    # 分批训练
    for i,data in enumerate(train_loader):
        # 清空梯度
        optimizer.zero_grad()
        # 前向传播，可以计算出预测结果
        train_pred = model(data[0].cuda())
        # 反向传播，就是计算微分，即用于求loss/cost function的最小值，用于更新参数
        batch_loss=loss(train_pred,data[1].cuda())
        batch_loss.backward()
        # 紧跟着优化
        optimizer.step()
        # 计算预测精度，argmax：获得最大值的下标，此处对应可能性最大的分类，axis=1意思是合并该行上所有的列
        train_acc+=np.sum(np.argmax(train_pred.cpu().data.numpy(),axis=1)==data[1].numpy())
        train_loss+=batch_loss.item()
    model.eval()
    # torch.no_grad() 是一个上下文管理器，被该语句 wrap 起来的部分将不会track梯度，将不再计算张量的梯度，跟踪张量的历史记录。这点在评估模型、测试模型阶段中常常用到，减少资源的使用
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))